# Elasticsearch
엘라스틱서치의 기본 원리와 사용법에 대해서 배웁니다.
____

## 1. Elasticsearch의 Python 라이브러리 설치

In [ ]:
!pip install elasticsearch

In [ ]:
from elasticsearch import Elasticsearch

## 2. Index 세팅
- 한글 데이터를 다루기 때문에, 은전한닢(Mecab) 형태소 분석기를 탑재합니다.

In [ ]:
import pprint  
INDEX_NAME = "toy_index"


INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer"
          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "content" : {
          "type" : "text",
          "analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean"
        }
      }

  }
}


## 3. 문서 준비

In [ ]:
DOCS = {
    1: {"title": "My Love",
        "content": "사랑해 그 말은 무엇보다 아픈 말 숨죽여서 하는 말 이젠 하기 힘든 말"
        },
    2: {"title": "듣고있나요",
        "content": "끝내 우린 스쳐가나요 기억넘어 서로를 지워야하나요 내게 사랑이 준 깊은 상처는 어떻게 견디며 살아야하는지 매일 아픈 그리움속에 가슴 텅 빈채 살아도 그대를 사랑했던일 그것만은 죽어도 나 후회하지않아요"
        },
    3: {"title": "인연",
        "content": "눈을 떠 바라보아요 그댄 정말 가셨나요 단 한번 보내준 그대 눈빛은 날 사랑했나요 또 다른 사랑이 와도 이젠 쉽게 허락되진 않아 견디기 힘들어 운명같은 우연을"
        },
    4: {"title": "말리 꽃",
        "content": ["얼마나 더 견뎌야 하는지 짙은 어둠을 헤메고 있어","내가 바란 꿈이라는 것은 없는걸까", "더 이상은 견딜수 없는 것"]
        },
    5: {"title": "그런 사람 또 없습니다",
        "content": "천번 번이고 다시 태어난대도 그런 사람 또 없을테죠 슬픈 내삶을 따뜻하게 해준 잠 고마운 사람입니다"
        }
}  

## 4. Elastichsearch 접속

In [ ]:

try:
    es.transport.close()
except:
    pass
es = Elasticsearch()



In [ ]:
es.info()

## 5. 인덱스 생성

In [ ]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

## 6. 문서삽입

In [ ]:
import time
for doc_id, doc in DOCS.items():
    es.index(index=INDEX_NAME,  id=doc_id, body=doc)
    time.sleep(1)


## 7. 삽입된 문서 확인

In [ ]:
doc = es.get(index=INDEX_NAME, id=1)
pprint.pprint(doc)

## 8. Term vector 확인

In [ ]:
tv = es.termvectors(index=INDEX_NAME, id=2,body={"fields" : ["content","title"]})

In [ ]:
pprint.pprint(tv)

## 7. 검색

In [ ]:
query = "사랑하지만 힘들어 죽겠네"
res = es.indices.analyze(index=INDEX_NAME,
                                 body={
                                       "tokenizer" : "nori_tokenizer",
                                        "text" : query
                                 }
                        )
pprint.pprint(res)

In [ ]:
res = es.search(index=INDEX_NAME, q=query, size=10)
pprint.pprint(res)

In [ ]:
for hit in res['hits']['hits']:
    print("Doc ID: %3r  Score: %5.2f" % (hit['_id'], hit['_score']))
